<a href="https://colab.research.google.com/github/ben854719/Alarm-System_Cybersecurity/blob/main/Malware_Detection_in_Agentic_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade google-ai-generativelanguage==0.6.18
!pip install --upgrade langchain-google-genai google-generativeai LangGraph

  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl (1.4 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


  Using cached langchain_google_genai-2.1.5-py3-none-any.whl.metadata (5.2 kB)
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.4-py3-none-any.whl.metadata (4.2 kB)
  Using cached google_generativeai-0.8.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.2-py3-none-any.whl.metadata (3.9 kB)
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.0-py3-none-any.whl.metadata (3.9 kB)
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://

In [3]:
from IPython import get_ipython
from IPython.display import display
import os
from langgraph.graph import StateGraph
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import SystemMessage
from typing import TypedDict, List
from google.colab import userdata
from langchain_core.messages import HumanMessage
from langchain_core.runnables.config import RunnableConfig

api_key = userdata.get("Ben85")
if not api_key:
    raise ValueError("Ben85 secret not found. Please set your API key in Colab Secrets with the name 'Ben85'.")

# Define the state schema using TypedDict
class LogAnalysisState(TypedDict):
    logs: List[str] # Input: a list of log strings
    analysis: str # Output: a single string containing the analysis

# Initialize Gemini model
gemini_model = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest", api_key=api_key)

# Create a analyze log.
def analyze_logs(state: LogAnalysisState) -> dict:
    """
    Analyzes a list of security logs to detect anomalies using the Gemini model.

    Args:
        state: The current state of the LangGraph workflow, containing the logs.

    Returns:
        A dictionary containing the analysis to update the state. The key 'analysis'
        will hold a string with the detailed analysis, potentially including identified
        anomalies and their descriptions.
    """
    logs = state['logs']

    # Construct a more detailed prompt for the model.
    # Request a structured output if possible, or clear descriptions of anomalies.
    prompt = (
        "Analyze the following security logs carefully. Your task is to identify "
        "any anomalies, suspicious patterns, or potential security threats. "
        "For each identified anomaly, provide a brief description and indicate "
        "which log entries are related.\n\n"
        "Security Logs:\n" + "\n".join(logs) + "\n\n"
        "Please provide your analysis in a clear and concise manner."
    )

    # Invoke the Gemini model with the HumanMessage containing the prompt.
    try:
        response = gemini_model.invoke([HumanMessage(content=prompt)])
        analysis_result = response.content
    except Exception as e:
        analysis_result = f"Error during analysis: {e}"
        print(analysis_result) # Print error for debugging

    # Return a dictionary with the analysis content to update the state.
    return {"analysis": analysis_result}

# Create LangGraph workflow
workflow = StateGraph(state_schema=LogAnalysisState)
workflow.add_node("log_analysis", analyze_logs)
workflow.set_entry_point("log_analysis")

# Compile the workflow
app = workflow.compile()

# Create a security log.
security_logs = [
    "User login successful",
    "Suspicious activity detected in network traffic",
    "File accessed by unauthorized user",
    "System reboot initiated",
    "Failed login attempt from unknown IP address 192.168.1.100",
    "Regular system health check passed",
]

# Run the workflow
print("Running log analysis workflow...")
result = app.invoke({"logs": security_logs})
print("\nAnalysis Result:")
print(result['analysis'])


Running log analysis workflow...

Analysis Result:
* **Unauthorized File Access:** The entry "File accessed by unauthorized user" indicates a potential security breach.  This requires immediate investigation to determine the accessed file, the unauthorized user involved, and the extent of the compromise.

* **Suspicious Network Traffic:** Coupled with the unauthorized file access, "Suspicious activity detected in network traffic" strengthens the possibility of a security incident.  These two logs likely relate to the same event, suggesting data exfiltration or malware activity.

* **Failed Login from Unknown IP:** "Failed login attempt from unknown IP address 192.168.1.100" points to a possible brute-force attack or unauthorized access attempt.  While a single failed login isn't conclusive, it warrants monitoring for further attempts from this IP address.  It's worth noting that 192.168.1.100 is a private IP address, suggesting the attempt might originate from within the local network.